Passo a passo:
1. Receber os logs e modelos como na v1
2. Converter o log de atividades para o novo formato, trocando a coluna lifecycle por start e end no nome da atividade
3. Converter o log de acesso para o mesmo formato do log de atividades, colocando a operação no nome 
4. Adaptar o modelo declare para que todas as operações obrigatórias e proibidas do modelo de acesso virem regras declare

**Mudei o nome da atividade pra não ter traço e tentei usar Precendence e Response pra andar com a coisa, mas travei naquela questão do id da atividade, porque preciso saber de qual atividade é o acesso**

In [ ]:
from Declare4Py.ProcessModels.DeclareModel import DeclareModel

def check_letters(cell, modelo, acesso, atividade):
    """Verifica quais letras (c, r, u, d) estão presentes na célula, diferenciando maiúsculas e minúsculas."""
    # Letras a serem verificadas
    letters = ['c', 'r', 'u', 'd']
    
    if atividade == 'Ferramenta':
        return modelo
    
    cell = str(cell)  # Garantir que a célula é string
    for letter in letters:
        uppercase_present = letter.upper() in cell
        
        if uppercase_present:
            for d in range(1,2001):  
                if f'{acesso} {letter} {d}\n' not in modelo:
                    modelo += 'activity ' + f'{acesso} {letter} {d}\n'
            for d in range(1,2001):
                modelo += f'Precedence[{atividade} begin {d}, {acesso} {letter} {d}] | | |\n'
                modelo += f'Response[{acesso} {letter} {d}, {atividade} complete {d}] | | |\n'
    
    return modelo
    

def convertModelToRules(modeloAcesso, modeloProcesso):
    declare_model_activities = modeloProcesso.activities
    declare_model_constraints = modeloProcesso.serialized_constraints
    
    novoModelo = ''
    
    for act in declare_model_activities:
        for d in range(1,2001):
            novoModelo += 'activity ' + act + ' begin ' + str(d) + '\n'
            novoModelo += 'activity ' + act + ' complete ' + str(d) +'\n'

    # Criar um novo DataFrame com os resultados
    for col in modeloAcesso.columns:
        for index, value in modeloAcesso[col].items():
            novoModelo = check_letters(value, novoModelo, modeloAcesso.iloc[index,0], col)
            
    for rule in declare_model_constraints:
        if ',' in rule:
            rule1, rule2 = rule.split(',')
            rule2, rule3 = rule2.split(']')
            novoModelo += rule1 + ' begin,' + rule2 + ' begin' + ']' + rule3 + '\n'
        else:
            rule1, rule2 = rule.split(']')
            novoModelo += rule1 + ' begin' + ']' + rule2 + '\n'
    
    declare_model = DeclareModel().parse_from_string(novoModelo)
    declare_model.to_file('modeloRegrasConjunto.decl')
    return declare_model
  

In [ ]:
import pandas as pd
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.D4PyEventLog import D4PyEventLog
import pm4py

def PreProcessData(logProcessoPATH, logAcessoPATH, modeloRecursoPATH, modeloProcessoPATH, modeloAcessoPATH):
    logAcesso = D4PyEventLog(case_name="case:concept:name")
    logAcesso.parse_xes_log(logAcessoPATH)
    logAcessoProcessado = pm4py.convert_to_dataframe(logAcesso.get_log())
    logAcessoProcessado = logAcessoProcessado.sort_values(['case:concept:name', 'concept:instance'])
    modeloRecursoProcessado = pd.read_csv(modeloRecursoPATH, sep=';')
    modeloAcessoProcessado = pd.read_csv(modeloAcessoPATH, sep=';')
    declare_model = DeclareModel().parse_from_file(modeloProcessoPATH)
    event_log = D4PyEventLog(case_name="case:concept:name")
    event_log.parse_xes_log(logProcessoPATH)
    return event_log, logAcessoProcessado, modeloRecursoProcessado, declare_model, modeloAcessoProcessado
    

In [ ]:
import pm4py

def convertLogs(logProcesso, logAcesso):
    '''
    Converte os logs de processo e acesso em um log único que pode ser submetido a verificação de conformidade 
    por meio da biblioteca Declare4Py 
    '''
    logProcessoCSV = pm4py.convert_to_dataframe(logProcesso.get_log())
    logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance'])
    for index, row in logProcessoCSV.iterrows():
        logProcessoCSV.at[index, 'concept:name'] = row['concept:name'] + ' ' + row['lifecycle:transition'] + ' ' + str(row['@@case_index']+1)
    logProcessoCSV.drop('lifecycle:transition', axis=1, inplace=True)
    
    for index, row in logAcesso.iterrows():
        logProcessoCSV.loc[len(logProcessoCSV)] = [(row['concept:tool'] + ' ' + row['concept:operation'].lower()) + ' ' + str(row['@@case_index']+1),row['time:timestamp'], row['concept:resource'], row['concept:instance'], len(logProcessoCSV), row['@@case_index'], row['case:concept:name']]
    logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance','time:timestamp'])
    logProcessoCSV.to_csv('LogTesteConjunto.csv', index=False)
    pm4py.write_xes(logProcessoCSV, "LogSinteticoConjuntoOFICIAL.xes")

In [ ]:
def formatViolations(df_violations):
    '''
    Formata as violações encontradas na verificação de conformidade do Declare4Py
    '''
    violacoes = []
    for index, row in df_violations.iterrows():
        for coluna in df_violations.columns:
            if row[coluna] == 1:
                violacoes.append(f"Trace {index} violou {coluna}")
    return violacoes

In [ ]:
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser

def CheckProcessConformance(modeloProcesso):
    '''
    Checa conformidade de processo entre o log e o modelo DECLARE com a biblioteca Declare4Py
    Recebe um log de processo (em formato EventLog) e um modelo de processo (em DeclareModel)
    
    '''
    event_log = D4PyEventLog(case_name="case:concept:name")
    event_log.parse_xes_log("LogSinteticoConjuntoOFICIAL.xes")
    
    model_constraints = modeloProcesso.get_decl_model_constraints()

    # print("Model constraints:")
    # print("-----------------")
    # for idx, constr in enumerate(model_constraints):
    #     print(idx, constr)
    basic_checker = MPDeclareAnalyzer(log=event_log, declare_model=modeloProcesso, consider_vacuity=False)
    conf_check_res: MPDeclareResultsBrowser = basic_checker.run()
    violacoes = formatViolations(conf_check_res.get_metric(metric="num_violations"))
    return violacoes

In [ ]:
import pm4py

def CheckResourceConformance(logProcesso, logAcesso, modeloRecurso):
    '''
    Checa conformidade entre o log de acesso, log de processo e o modelo de recurso
    Recebe modelo de recurso e logs em csv
    '''
    
    logProcessoCSV = pm4py.convert_to_dataframe(logProcesso.get_log())
    logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance'])
    logProcessoFiltrado = logProcessoCSV[logProcessoCSV["lifecycle:transition"] == "begin"]
    
    violacoes = {}
    violacoes["AcessoRecursoEquipe"] = []
    violacoes["AcessoRecursoErrado"] = []
    violacoes["Atividade"] = []
    violacoes["OperacaoProibida"] = [] 
    violacoes["AcessoObgNaoRealizado"] = [] 
    acessosPermitidos = ['c', 'r', 'u', 'd']
    for index, row in modeloRecurso.iterrows():
        demanda = row['case:concept:name']
        recursos = row['concept:resources'].split(", ")
        atividades = logProcessoFiltrado[logProcessoFiltrado['case:concept:name'] == demanda]
        acessos = logAcesso[logAcesso['case:concept:name'] == demanda]
        for i, atv in atividades.iterrows():
            recursoAtv = atv['concept:resource']
            if recursoAtv not in recursos:
                violacoes["Atividade"].append([atv['concept:name'], demanda, recursoAtv])
            
            acessosAtv = acessos[acessos['concept:instance']== atv['concept:instance']]
            for j, acc in acessosAtv.iterrows():
                if recursoAtv != acc['concept:resource']:
                    violacoes["AcessoRecursoErrado"].append([acc['concept:tool'], demanda, acc['concept:resource'], recursoAtv])
                if acc['concept:resource'] not in recursos:
                    violacoes["AcessoRecursoEquipe"].append([acc['concept:tool'], demanda, acc['concept:resource']])
            
            cell = str(acc['concept:operation'])
            for letter in cell:
                if letter.lower() and letter not in acessosPermitidos:
                    violacoes["OperacaoProibida"].append([atv['concept:name'], demanda, letter, acc['concept:tool']])
                if letter.upper() and letter.lower() not in cell.lower():
                    violacoes["AcessoObgNaoRealizado"].append([atv['concept:name'], demanda, letter, acc['concept:tool']])
    return violacoes

In [ ]:
def formatInconformances(conformanceProcess, conformanceResource, conformanceAcess):
    '''
    Formata as inconformidades encontradas em cada verificação (pode vir a combinar os padrões de anomalias no futuro)
    '''
    print('Violações de fluxo de processo:')
    for violation in conformanceProcess:
        print(violation)
    
    print("Violações de Acesso:")
    for key, violation in conformanceAcess.items():
        if key == "OperacaoProibida":
            for occur in violation:
                print(f'Violação de Acesso na atividade {occur[0]} no trace {occur[1]}, a operação {occur[2]} não era permitida para a ferramenta {occur[3]}')
        if key == "AcessoObgNaoRealizado":
            for occur in violation:
                print(f"Violação de Acesso na atividade {occur[0]} no trace {occur[1]}, os acessos obrigatórios às ferramentas {occur[2]} não foram realizados")
    
    
    print('Violações de Privacidade:')
    for key, violation in conformanceResource.items():
        if key == "Atividade":
            for occur in violation:
                print(f'Violação de Privacidade na atividade {occur[0]} no trace {occur[1]}, o recurso {occur[2]} não faz parte da equipe determinada para realizar a demanda')
        if key == "AcessoRecursoEquipe":
            for occur in violation:
                print(f'Violação de Privacidade no acesso a {occur[0]} no trace {occur[1]}, o recurso {occur[2]} não faz parte da equipe determinada para realizar a demanda')
        if key == "AcessoRecursoErrado":
            for occur in violation:
                print(f'Violação de Privacidade no acesso a {occur[0]} no trace {occur[1]}, o recurso {occur[2]} não era o designado da atividade vinculada e sim o {occur[3]}')
    

In [ ]:
def MultiperspectiveConformanceAlgorithm(logProcessoPATH, logAcessoPATH, modeloRecursoPATH, modeloProcessoPATH, modeloAcessoPATH):
  '''
  O algoritmo recebe: um log de processo, um log de acesso a dados, um modelo de recurso, um modelo DECLARE de processo e um modelo de acesso a dados
  '''
  logProcesso, logAcesso, modeloRecurso, modeloProcesso, modeloAcesso = PreProcessData(logProcessoPATH, logAcessoPATH, modeloRecursoPATH, modeloProcessoPATH, modeloAcessoPATH)
  modeloAcessoConvertido = convertModelToRules(modeloAcesso, modeloProcesso)
  convertLogs(logProcesso, logAcesso)
  conformanceProcess = CheckProcessConformance(modeloAcessoConvertido)
  conformanceResource = CheckResourceConformance(logProcesso, logAcesso, modeloRecurso)
  return formatInconformances(conformanceProcess, conformanceResource, conformanceAcess)

In [ ]:
MultiperspectiveConformanceAlgorithm('./LogSinteticoProcessoOFICIAL.xes', './LogSinteticoAcessoOFICIAL.xes', './ModeloRecursosOFICIAL.csv', './Modelo_Log_Sintetico_OFICIAL.decl', './ModeloAcessoOFICIAL.csv')

parsing log, completed traces :: 100%|██████████| 2000/2000 [00:16<00:00, 119.19it/s]


Violações de fluxo de processo:
Trace 0 violou Precedence[Manutencao de funcionalidade begin 1, Codigo u 1] | | |
Trace 0 violou Response[Codigo u 1, Manutencao de funcionalidade complete 1] | | |
Trace 0 violou Response[Requisito u 1, Atualizacao de requisitos funcionais complete 1] | | |
Trace 0 violou Response[Codigo c 1, Construcao de funcionalidade complete 1] | | |
Trace 0 violou Precedence[Documentacao de requisitos funcionais begin 1, Gestao u 1] | | |
Trace 0 violou Response[Gestao u 1, Contagem de ponto de funcao complete 1] | | |
Trace 1 violou Precedence[Manutencao de funcionalidade begin 2, Codigo u 2] | | |
Trace 1 violou Response[Codigo u 2, Manutencao de funcionalidade complete 2] | | |
Trace 1 violou Response[Requisito u 2, Atualizacao de requisitos funcionais complete 2] | | |
Trace 1 violou Precedence[Documentacao de requisitos funcionais begin 2, Gestao u 2] | | |
Trace 1 violou Response[Gestao u 2, Contagem de ponto de funcao complete 2] | | |
Trace 2 violou Respon